In [1]:
from datetime import datetime, timezone, timedelta
import numpy as np
import pandas as pd
import requests
from typing import NamedTuple
import xarray as xr

## Helper Functions

In [2]:
class SAMPLE(NamedTuple):
    val: int or float
    val_time: datetime
    val_lat: float
    val_lon: float
    
    
def get_skq_pco2(bdt: datetime, edt:datetime, nth: int = 1):
    params = {'model': 'BinnedDefaultFlowRolling',
              'decfactr': nth,
              'date_0': bdt.strftime('%Y-%m-%d %H:%M:%S'),
              'date_1': edt.strftime('%Y-%m-%d %H:%M:%S'),
              'format': 'json'}
    url = 'https://coriolix.sikuliaq.alaska.edu/api/decimateData'
    response = requests.get(url, params=params)
    if response.status_code == requests.codes.ok:
        data = response.json()
        ds_list = []
        for bin in data:
            time = bin['datetime_center']
            lat = bin['latitude']
            lon = bin['longitude']
            pco2_mean = bin['parameter_27']['b'][0]
            pco2_med = bin['parameter_27']['b'][-1]
            _ds = xr.Dataset()
            _ds = _ds.assign_coords({'time': [pd.to_datetime(time,format='mixed')]})
            _ds['latitude'] = (['time'],[lat])
            _ds['longitude'] = (['time'],[lon])
            _ds['pco2_mean'] = (['time'],[pco2_mean])
            _ds['pco2_med'] = (['time'],[pco2_med])
            ds_list.append(_ds)
        ds = xr.combine_by_coords(ds_list)
        ds['time'] = ds['time'].astype('datetime64[ns]')
        return ds
    else:
        raise ConnectionError(response.reason)

## Build QARTOD Tests

In [3]:
class QARTOD:
    
    class FLAG:
        PASS: int = 1
        NOT_EVALUATED: int = 2
        HIGH_INTEREST: int = 3
        SUSPECT: int = 3
        FAIL: int = 4
        MISSING: int = 9
     
    def __init__(self):
        pass
    
    def test_location(self, SAMPLE: object):
        
        # TODO: Implement condition for suspect flagging
        
        if abs(SAMPLE.val_lat) > 90 or abs(SAMPLE.val_lon) > 180:
            return self.FLAG.FAIL
        else:
            return self.FLAG.PASS
    
    
    def test_gross_range(self, SAMPLE: object, sensor_min: float, sensor_max: float, user_min: float or None = None, user_max: float or None = None) -> int:
        if np.isnan(sensor_min):
            return self.FLAG.NOT_EVALUATED
        elif np.isnan(sensor_max):
            return self.FLAG.NOT_EVALUATED
        if np.isnan(SAMPLE.val):
            return self.FLAG.MISSING
        if sensor_min < SAMPLE.val < sensor_max:
            flag = self.FLAG.PASS
        else:
            return self.FLAG.FAIL
        if user_min is not None and user_max is None: # Condition if only user_min is supplied.
            if SAMPLE.val < user_min:
                flag = self.FLAG.SUSPECT
        elif user_min is None and user_max is not None: # Condition if only user_max is supplied.
            if SAMPLE.val > user_max:
                flag = self.FLAG.SUSPECT
        elif user_min is not None and user_max is not None: # Condition if both user_min and user_max are supplied.
            if not user_min <= SAMPLE.val <= user_max: 
                flag = self.FLAG.SUSPECT
        return flag
            
    def test_climatology_month(self, SAMPLE: object, lookup: xr.Dataset, min_var_name, max_var_name) -> int:
        if np.isnan(SAMPLE.val) or np.isnat(np.datetime64(SAMPLE.val_time)) or np.isnan(SAMPLE.val_lat) or np.isnan(SAMPLE.val_lon):
            return self.FLAG.MISSING
        lookup = lookup.sel(month = SAMPLE.val_time.month , lat = SAMPLE.val_lat, lon = SAMPLE.val_lon, method = 'nearest')[[min_var_name, max_var_name]]
        if np.isnan(lookup[min_var_name]) or np.isnan(lookup[max_var_name]):
            return self.FLAG.NOT_EVALUATED
        else:
            if not lookup[min_var_name] <= SAMPLE.val <= lookup[max_var_name]:
                return self.FLAG.SUSPECT
            else:
                return self.FLAG.PASS
        
    
        

## Instantiate custom QARTOD Class

In [4]:
qrtd = QARTOD()

# Build Seasonal Lookup 

In [5]:
%%time
filepath = 'tests/test_data/SOCATv2024_tracks_gridded_monthly.nc'

# Import dataset and rename some variables.
ds = xr.open_dataset(filepath)
rename_vars = {'xlon': 'lon',
               'ylat': 'lat',
               'tmnth': 'time'}
ds = ds.rename(rename_vars)

ds = ds.sel(time = slice(datetime(2000,1,1),None)) # Slice dataset from 2000 onward.

# Create minimum value climatology for fCO2.
min_fco2 = ds['fco2_min_unwtd'].groupby('time.month').min(skipna=True)
max_fco2 = ds['fco2_max_unwtd'].groupby('time.month').max(skipna=True)

# Create minimum value climatology for SST.
min_sst = ds['sst_min_unwtd'].groupby('time.month').min(skipna=True)
max_sst = ds['sst_max_unwtd'].groupby('time.month').max(skipna=True)

# Create minimum value climatology for salinity.
min_sal = ds['salinity_min_unwtd'].groupby('time.month').min(skipna=True)
max_sal = ds['salinity_max_unwtd'].groupby('time.month').max(skipna=True)

# Merge datasets.
qartod_seasonals = xr.combine_by_coords([min_fco2, max_fco2, min_sst, max_sst, min_sal, max_sal])

CPU times: total: 4.73 s
Wall time: 4.78 s


## Acquire Last 10 Minutes of SKQ PCO2 Data

In [6]:
edt = datetime.now(timezone.utc)
bdt = edt - timedelta(seconds = 60 * 10)
skq_pco2 = get_skq_pco2(bdt,edt)

## Flag Data

In [7]:
%%time
dts = skq_pco2.time.values
for dt in dts:
    _ds = skq_pco2.sel(time = dt)
    sample = SAMPLE(val = float(_ds.pco2_mean.values),val_time=pd.to_datetime(_ds.time.values).to_pydatetime(),val_lat = float(_ds.latitude.values),val_lon = float(_ds.longitude.values))
    qr_clim = qrtd.test_climatology_month(sample, lookup = qartod_seasonals,min_var_name = 'fco2_min_unwtd', max_var_name = 'fco2_max_unwtd')
    qr_gross = qrtd.test_gross_range(sample, sensor_min = 0, sensor_max = 2000, user_min = 249, user_max = 1500)
    
    print(f"Val: {sample.val}, Climatology Test Flag: {qr_clim}, Gross Range Flag: {qr_gross}")

Val: -999.0, Climatology Test Flag: 2, Gross Range Flag: 4
Val: 248.77, Climatology Test Flag: 2, Gross Range Flag: 3
Val: -999.0, Climatology Test Flag: 2, Gross Range Flag: 4
Val: -999.0, Climatology Test Flag: 2, Gross Range Flag: 4
Val: 249.25, Climatology Test Flag: 2, Gross Range Flag: 1
Val: -999.0, Climatology Test Flag: 2, Gross Range Flag: 4
Val: 249.5, Climatology Test Flag: 2, Gross Range Flag: 1
Val: -999.0, Climatology Test Flag: 2, Gross Range Flag: 4
CPU times: total: 15.6 ms
Wall time: 10 ms


In [18]:
lookup = qartod_seasonals.sel(month = sample.val_time.month, lat = sample.val_lat, lon = sample.val_lon,method = 'nearest')

In [19]:
lookup

<xarray.Dataset> Size: 48B
Dimensions:             ()
Coordinates:
    lon                 float64 8B -162.5
    lat                 float64 8B 53.5
    month               int64 8B 6
Data variables:
    fco2_max_unwtd      float32 4B nan
    fco2_min_unwtd      float32 4B nan
    salinity_max_unwtd  float32 4B nan
    salinity_min_unwtd  float32 4B nan
    sst_max_unwtd       float32 4B nan
    sst_min_unwtd       float32 4B nan

In [23]:
qartod_seasonals.sel(month = sample.val_time.month, lat = 52.5, lon = sample.val_lon,method = 'nearest')

<xarray.Dataset> Size: 48B
Dimensions:             ()
Coordinates:
    lon                 float64 8B -162.5
    lat                 float64 8B 52.5
    month               int64 8B 6
Data variables:
    fco2_max_unwtd      float32 4B 355.2
    fco2_min_unwtd      float32 4B 256.0
    salinity_max_unwtd  float32 4B 33.0
    salinity_min_unwtd  float32 4B 31.88
    sst_max_unwtd       float32 4B 9.223
    sst_min_unwtd       float32 4B 7.1